## Import necessary libraries and load model

In [ ]:
## Give appropriate permission to the directory "FOLDER_WITH_ARTIFACTS" you are working with
import os
os.environ['SNPE_ROOT']="/local/mnt/workspace/aditya/qaisw-v2.15.1.230926150623_62883"#set up your snpe path here.
os.environ['RAW_FILE_FOLDER']="input/raw"
os.environ['DLC32']="models/lraspp_fp32.dlc"
os.environ['DLC8']="models/lraspp_w8a16.dlc"
os.environ['TARGET_INPUT_LIST']="input/input.txt"
os.environ['ONDEVICE_FOLDER']="lraspp"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="2dce6316" #change with your device-id. Use command "adb devices" to get devices names.
os.environ['SNPE_TARGET_ARCH']="aarch64-android"
os.environ['SNPE_TARGET_STL']="libc++_shared.so"

In [ ]:
import os
import torchvision.models.segmentation as models
import torch
import torch.nn as nn

In [ ]:
pretrained_model = models.lraspp_mobilenet_v3_large(pretrained=True)
class CustomModel(nn.Module):
    def __init__(self,pretrained_model):
        super(CustomModel,self).__init__()
        self.pretrained_model = pretrained_model
        self.argmax = nn.LogSoftmax(dim=1)

    def forward(self,x):
        output_dict = self.pretrained_model(x)
        output = output_dict['out']
        argmax_output = torch.argmax(output,dim=1,keepdim=False)
        return argmax_output

model = CustomModel(pretrained_model)
input = torch.randn(1,3,400,400)
output = model(input)
output.shape

In [ ]:
os.makedirs('models',exist_ok=True)

In [ ]:
dummy_input = torch.randn(1,3, 400, 400).type(torch.FloatTensor).to('cpu')
torch.onnx.export(model, dummy_input, "./models/lraspp.onnx",opset_version=11)

In [ ]:
import os
import cv2
import glob
import numpy as np
import torch
from os.path import isfile, join
import matplotlib.pyplot as plt 
from PIL import Image
from torchvision import transforms as T

In [ ]:
os.makedirs('input',exist_ok=True)
os.makedirs('input/dataset',exist_ok=True)

## Download dataset in input/dataset/ directory

User needs to download dataset of their choice before proceeding further

In [ ]:
directory_path = 'input/raw/'
output_file_path = 'input/input.txt'  # The file where the output will be saved

all_files = os.listdir(directory_path)

# Filter only the .raw files and create a list of their names
raw_files = [file for file in all_files if file.endswith('.raw')]
raw_files = sorted(raw_files)
# Write the file names to the output file
with open(output_file_path, 'w') as f:
    c=0
    for raw_file in raw_files:
        f.write(f"./raw/{raw_file}\n")
        c=c+1
       

print(f"File names written to {output_file_path}.")

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-onnx-to-dlc --input_network models/lraspp.onnx --output_path models/lraspp_fp32.dlc


In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
cd input/
snpe-dlc-quantize --input_dlc ../models/lraspp_fp32.dlc --input_list input.txt  --axis_quant --output_dlc ../models/lraspp_mobilenet_v3_large_quant_w8a8.dlc  --enable_htp --htp_socs sm8550

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
cd input/
snpe-dlc-quantize --input_dlc ../models/lraspp_fp32.dlc --input_list input.txt  --axis_quant --output_dlc ../models/lraspp_w8a16.dlc  --act_bitwidth 16  --enable_htp 

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-dlc-graph-prepare --input_dlc models/lraspp_fp32.dlc --output_dlc models/lraspp_fp16.dlc --use_float_io 

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/dsp/lib"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/$SNPE_TARGET_STL /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
$DEVICE_SHELL push $SNPE_ROOT/bin/$SNPE_TARGET_ARCH/snpe-net-run /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin
$DEVICE_SHELL push $SNPE_ROOT/lib/hexagon-v75/unsigned/*.so /data/local/tmp/snpeexample/dsp/lib
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/*.so /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

In [ ]:
%%bash
#find ./raw -name *.raw > list.txt
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $DLC32 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $DLC8 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $RAW_FILE_FOLDER /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_32b_CPU
export OUTPUT_DLC_32=lraspp_fp32.dlc
export ONDEVICE_FOLDER="lraspp"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_32 --input_list input.txt --output_dir $OUTPUT_FOLDER 

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_8b_DSP
export OUTPUT_DLC_QUANTIZED8=lraspp_w8a16.dlc
export ADSP_LIBRARY_PATH="/data/local/tmp/snpeexample/dsp/lib;/system/lib/rfsa/adsp;/system/vendor/lib/rfsa/adsp;/dsp"
export ONDEVICE_FOLDER="lraspp"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
# modified the inputlist.txt. That's the only change
snpe-net-run --container $OUTPUT_DLC_QUANTIZED8 --input_list input.txt --output_dir $OUTPUT_FOLDER --use_dsp 

In [ ]:
rm -rf output/

In [ ]:
os.makedirs('output', exist_ok=True)

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_8b_DSP output/OUTPUT_8b_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_32b_CPU output/OUTPUT_32b_CPU

In [ ]:
def image_overlay(image, segmented_image):
    alpha = 1 # transparency for the original image
    beta = 0.8 # transparency for the segmentation map
    gamma = 0 # scalar added to each sum
    # print(image.size)
    # print(segmented_image.shape)
    segmented_image = cv2.cvtColor(segmented_image, cv2.COLOR_RGB2BGR)
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.addWeighted(image, alpha, segmented_image, beta, gamma, image)
    return image

In [ ]:
import torchvision.transforms as transforms
crop_size = 400
transform = transforms.Compose([
    transforms.Resize((crop_size,crop_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
label_map = [
               (0, 0, 0),  # background
               (128, 0, 0), # aeroplane
               (0, 128, 0), # bicycle
               (128, 128, 0), # bird
               (0, 0, 128), # boat
               (128, 0, 128), # bottle
               (0, 128, 128), # bus 
               (128, 128, 128), # car
               (64, 0, 0), # cat
               (192, 0, 0), # chair
               (64, 128, 0), # cow
               (192, 128, 0), # dining table
               (64, 0, 128), # dog
               (192, 0, 128), # horse
               (64, 128, 128), # motorbike
               (192, 128, 128), # person
               (0, 64, 0), # potted plant
               (128, 64, 0), # sheep
               (0, 192, 0), # sofa
               (128, 192, 0), # train
               (0, 64, 128) # tv/monitor
]

In [ ]:
def get_segment_labels(image, model, device):
    # transform the image to tensor and load into computation device
    image = transform(image).to(device)
    image = image.unsqueeze(0) # add a batch dimension
    outputs = model(image)
    return outputs

In [ ]:
def draw_segmentation_map(outputs):
    labels = outputs.detach().cpu().numpy()
    # create Numpy arrays containing zeros
    # later to be used to fill them with respective red, green, and blue pixels
    red_map = np.zeros_like(labels).astype(np.uint8)
    green_map = np.zeros_like(labels).astype(np.uint8)
    blue_map = np.zeros_like(labels).astype(np.uint8)
    
    for label_num in range(0, len(label_map)):
        index = labels == label_num
        red_map[index] = np.array(label_map)[label_num, 0]
        green_map[index] = np.array(label_map)[label_num, 1]
        blue_map[index] = np.array(label_map)[label_num, 2]
        
    segmentation_map = np.stack([red_map, green_map, blue_map], axis=2)
    return segmentation_map

In [ ]:
def ImageNames():
    inputlist = open('input/input.txt', 'r')
    Lines = inputlist.readlines()
    count = 0
    imageList = []
    for line in Lines:
        name = line.split("preproc_",1)[1]
        name = name.split('.')[0]
        imageList.append(name)
        count += 1
    return imageList
imageList = ImageNames()
print((imageList))

In [ ]:
import torch
import numpy as np
device = torch.device('cpu')
model.eval()

In [ ]:
os.makedirs('output/model_prediction', exist_ok=True)

In [ ]:
image_dir = # mention dataset path here
from PIL import Image
import cv2
for i in range(0,len(imageList)):

    image = Image.open(image_dir+imageList[i]+'.jpg')
    # do forward pass and get the output dictionary
    image  = image.resize((crop_size,crop_size))
    # print(image.size)
    outputs = get_segment_labels(image, model, device)
    # get the data from the `out` key
    # outputs = outputs['out']
    # print(type(outputs))
    # print(outputs.shape)
    segmented_image = draw_segmentation_map(outputs[0])
    print(image.size)
    print(segmented_image.shape)
    final_image = image_overlay(image, segmented_image)
    # show the segmented image and save to disk
    # cv2.imshow('Segmented image', final_image)
    # cv2.waitKey(0)
    cv2.imwrite(f"output/model_prediction/{imageList[i]}.jpg", final_image)

In [ ]:
def PostProc(img_path, out_path,i):
    res = np.fromfile(img_path, dtype="float32")
    res_reshape = res.reshape((1,400,400)).astype(np.float32)
    model_img = torch.from_numpy(res_reshape)
    segmented_image = draw_segmentation_map(model_img[0])
    image = Image.open(image_dir+imageList[i]+'.jpg')
    # do forward pass and get the output dictionary
    image  = image.resize((crop_size,crop_size))
    final_image = image_overlay(image, segmented_image)
    cv2.imwrite(f"{out_path}", final_image)    

In [ ]:
os.makedirs('output/test_results', exist_ok=True)
os.makedirs('output/test_results/32b_arm', exist_ok=True)
os.makedirs('output/test_results/8b_dsp', exist_ok=True)

In [ ]:
test_images_dir = "output/OUTPUT_32b_CPU/"
image_dir = # mention dataset path here
import cv2
import os
import numpy as np
import torch
from PIL import Image
for i in range(0,len(imageList)):
    img_path = os.path.join(test_images_dir, 'Result_')
    img_path = img_path+str(i)+'/732.raw'
    out_path = 'output/test_results/32b_arm/'+imageList[i]+'_prediction_32b_arm.png'
    PostProc(img_path, out_path,i)
    i = i +1

In [ ]:
test_images_dir = "output/OUTPUT_8b_DSP/"
for i in range(0,len(imageList)):
    img_path = os.path.join(test_images_dir, 'Result_')
    img_path = img_path+str(i)+'/732.raw'
    out_path = 'output/test_results/8b_dsp/'+imageList[i]+'_prediction_8b_dsp.png'
    PostProc(img_path, out_path,i)
    i = i +1

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(30, 100));
import cv2
for i in range(0, 5):
    
    original = cv2.imread('<mention dataset path here>'+imageList[i]+'.jpg')
    original = cv2.resize(original, (513,513))
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    ax = fig.add_subplot(28,4,4*i+1);
    plt.imshow(original,cmap='gray');
    ax.set_title('original image\n');
    ax.axis('off');
    
    pth_inf = cv2.imread('output/model_prediction/'+imageList[i]+'.jpg')
    pth_inf = cv2.resize(pth_inf, (513,513))
    # pth_inf = cv2.cvtColor(pth_inf, cv2.COLOR_BGR2RGB)
    # pth_overlay = image_overlay(original, pth_inf)
    ax = fig.add_subplot(28,4,4*i+2);
    plt.imshow(pth_inf,cmap='gray');
    ax.set_title('pth output\n');
    ax.axis('off');


    arm_fp32= cv2.imread('output/test_results/32b_arm/'+imageList[i]+'_prediction_32b_arm.png')
    # arm_fp32 = cv2.cvtColor(arm_fp32, cv2.COLOR_BGR2RGB)
    # fp32_overlay = image_overlay(original, arm_fp32)
    arm_fp32 = cv2.resize(arm_fp32, (513,513))
    ax = fig.add_subplot(28,4,4*i+3);
    plt.imshow(arm_fp32,cmap='gray');
    ax.set_title('fp32 on ARM\n');
    ax.axis('off');

    dsp= cv2.imread('output/test_results/8b_dsp/'+imageList[i]+'_prediction_8b_dsp.png')
    # dsp_int8 = cv2.cvtColor(dsp_int8, cv2.COLOR_BGR2RGB)
    # int8_overlay = image_overlay(original, dsp_int8)
    # print(dsp)
    ax = fig.add_subplot(28,4,4*i+4);
    plt.imshow(dsp,cmap='gray');
    ax.set_title('int8 on DSP\n');
    ax.axis('off');